## Set up LLaMa

In [1]:
from langchain_community.llms import LlamaCpp
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="llama_models\llama-2-7b-chat.Q5_K_S.gguf",
    temperature=0,
    max_tokens=500,
    n_ctx=3900,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama_models\llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32   

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Prepare PDF documents

### Make documents from PDFs

In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_paths = os.scandir('resources/all_pdfs/')
pages = []
for path in pdf_paths:
    if path.is_file():
        loader = PyPDFLoader('resources/all_pdfs/' + path.name)
        new_pages = loader.load_and_split()
        pages.extend(new_pages)

In [5]:
print(pages[0])

page_content='Department of Computer Science\nCOS 132\nImperative Programming\nLecturers: Ms. Tayana Morkel, Dr. Patricia Lutu and Dr. Vreda Pieterse\nc⃝Copyright reserved\n1' metadata={'source': 'resources/all_pdfs/COS 132 study guide.pdf', 'page': 0}


Generate meta-information from first chunk of each document

In [6]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

metadata_generation_template = """\
<<SYS>>

What is the six-character module code for this module in the study guide below? Provide those six characters only - do not provide any other information. No yapping.

Format the response as the three letters of the code, and then the three numbers, for example: COS 999

<</SYS>>

[INST]
Extract the module code from the below text.

study guide snippet: {study_guide_snippet}

[/INST]
"""

metadata_generation_prompt_template = PromptTemplate.from_template(metadata_generation_template)

metadata_chain = (
    {"study_guide_snippet": RunnablePassthrough()}
    | metadata_generation_prompt_template
    | llm
    | StrOutputParser()
)

In [7]:
import pickle
saved_metadata = dict()
if os.path.exists('generated_metadata/saved_metadata.pkl'):
    with open('generated_metadata/saved_metadata.pkl', 'rb') as f:
        saved_metadata = pickle.load(f)
        print("Loaded saved data!")
        print(saved_metadata)

for page in pages:
    generated_metadata = ""
    page.page_content = str(page.page_content).replace('\n', ' ')
    page.page_content = str(page.page_content).replace('.', ' ')
    page.page_content = str(page.page_content).replace("'", ' ')  
    page.page_content = str(page.page_content).replace("`", ' ')  

    # for the "root" page of each document
    if page.metadata['page'] == 0:
        # set generated metadata to either the saved metadata...
        if saved_metadata.keys() != None and page.metadata['source'] in saved_metadata.keys():
            generated_metadata = saved_metadata[page.metadata['source']]
        # or generate new metadata if there is no saved metadata
        else:
            generated_metadata = metadata_chain.invoke(page.page_content)
            saved_metadata[page.metadata['source']] = generated_metadata
        
        # loop through all documents...
for page in pages:
    # update content to add metadata context.
    page.page_content = 'Module ' + saved_metadata[page.metadata['source']] + ':\n' + str(page.page_content) + '\n\n'


Loaded saved data!
{'resources/all_pdfs/COS 132 study guide.pdf': 'COS 132', 'resources/all_pdfs/COS110-studyGuide.pdf': 'COS 110', 'resources/all_pdfs/COS122_StudyGuide.pdf': 'COS 122', 'resources/all_pdfs/COS151_study_guide.pdf': 'COS 151', 'resources/all_pdfs/COS212_study_guide.pdf': 'COS 212', 'resources/all_pdfs/COS214StudyGuide2020_V1_0(1).pdf': 'COS 214', 'resources/all_pdfs/COS216_StudyGuide.pdf': 'COS 216', 'resources/all_pdfs/COS710StudyGuide-2023.pdf': 'COS 710', 'resources/all_pdfs/study guide(1).pdf': 'FNAS', 'resources/all_pdfs/Study guide(2).pdf': 'The six-character module code for WTW 148 is: COS 999', 'resources/all_pdfs/study guide.pdf': 'The six-character module code for the module "Statistics (STK 220)" in the study guide is:\nSTK 220', 'resources/all_pdfs/study_guide.pdf': 'COS 999'}


In [8]:
import pprint
# Prints the nicely formatted dictionary
pprint.pprint(saved_metadata)

{'resources/all_pdfs/COS 132 study guide.pdf': 'COS 132',
 'resources/all_pdfs/COS110-studyGuide.pdf': 'COS 110',
 'resources/all_pdfs/COS122_StudyGuide.pdf': 'COS 122',
 'resources/all_pdfs/COS151_study_guide.pdf': 'COS 151',
 'resources/all_pdfs/COS212_study_guide.pdf': 'COS 212',
 'resources/all_pdfs/COS214StudyGuide2020_V1_0(1).pdf': 'COS 214',
 'resources/all_pdfs/COS216_StudyGuide.pdf': 'COS 216',
 'resources/all_pdfs/COS710StudyGuide-2023.pdf': 'COS 710',
 'resources/all_pdfs/Study guide(2).pdf': 'The six-character module code for '
                                          'WTW 148 is: COS 999',
 'resources/all_pdfs/study guide(1).pdf': 'FNAS',
 'resources/all_pdfs/study guide.pdf': 'The six-character module code for the '
                                       'module "Statistics (STK 220)" in the '
                                       'study guide is:\n'
                                       'STK 220',
 'resources/all_pdfs/study_guide.pdf': 'COS 999'}


In [9]:
with open('generated_metadata/saved_metadata.pkl', 'wb') as f:
    saved_metadata = pickle.dump(saved_metadata, f)

In [10]:
pprint.pprint(pages[30].page_content)

('Module COS 122:\n'
 'The ﬁnal EO is a written exam during the normal examination period  The '
 'following regula- tions hold: •To pass the course, a student must obtain a '
 'ﬁnal mark of at least 50%  •A student will pass the course with distinction '
 'if he/she pass the course with a ﬁnal mark of at least 75%  Note that all '
 'these mark thresholds are applied strictly  The marks of students who come '
 'close to them, but do not actually meet them will notbe condoned (i e  put '
 'up)  Please also take note of the examination rules, as provided for in the '
 'general rules and regulations of the University of Pretoria, under section G '
 '12  The date and venue for the exam is scheduled on the ofﬁcial UP exam time '
 'table  6 2 Mark Distribution Marks for all events will be published on the '
 'CS Website  Students are responsible for verifying the correctness of these '
 'marks  Please ﬁle complaints about marks via email, within 10 calendar days '
 'after publication, to the

## Create vector database from split documents

In [11]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(pages, hf_embeddings)

### Create a retrieval object which will be used in the query chain to provide context

In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [20]:
# Sanity check - should return COS122_StudyGuide.pdf
test_retrieval_input = """\
All practical work must be done in C++ and must compile and run in the Informatorium under
Linux, unless explicitly stated otherwise
"""
results_with_scores = db.similarity_search_with_score(test_retrieval_input, k=3)
for doc, score in results_with_scores:
    print(f"Metadata: {doc.metadata}, Score: {score}")

Metadata: {'source': 'resources/all_pdfs/COS110-studyGuide.pdf', 'page': 3}, Score: 0.8837088346481323
Metadata: {'source': 'resources/all_pdfs/COS 132 study guide.pdf', 'page': 5}, Score: 1.0808874368667603
Metadata: {'source': 'resources/all_pdfs/study_guide.pdf', 'page': 7}, Score: 1.205413818359375


## Make a template and chain with which to make queries

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

# Method for formatting documents to place into the context of a query
def format_docs(docs):
    return "\n\n".join(str(str(doc.metadata) + ":\n " + str(doc.page_content) + "\n next document:\n") for doc in docs)

study_guide_rag_template = """\
<<SYS>>

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

context: {context}

<</SYS>>

[INST]
question: {question}
answer:
[/INST]
"""

custom_rag_prompt = PromptTemplate.from_template(study_guide_rag_template)

retrieval_query_template = """\
<<SYS>>

Extract keywords from the given question.

Respond with a comma-seperated list of keywords ONLY. If a module code is found, prioritise it.

<</SYS>>

[INST]
question: {question}
keywords:
[/INST]
"""

retrieval_query_prompt_template = PromptTemplate.from_template(retrieval_query_template)

rag_chain = (
    {"context": retrieval_query_prompt_template | llm | retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [21]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

# response = rag_chain.invoke({"question": "What is Anna Bosman's email address?"}, config={'callbacks': [ConsoleCallbackHandler()]})
# response = rag_chain.invoke({"question": "What is Anna Bosman's email address?"})

Llama.generate: prefix-match hit


Here are the keywords extracted from your question:

* Anna Bosman
* email address


llama_print_timings:        load time =    1348.50 ms
llama_print_timings:      sample time =       3.98 ms /    20 runs   (    0.20 ms per token,  5022.60 tokens per second)
llama_print_timings: prompt eval time =    8312.85 ms /    72 tokens (  115.46 ms per token,     8.66 tokens per second)
llama_print_timings:        eval time =    2673.59 ms /    19 runs   (  140.72 ms per token,     7.11 tokens per second)
llama_print_timings:       total time =   11052.62 ms /    91 tokens
Llama.generate: prefix-match hit


I don't know Anna Bosman's email address as it is not provided in the given documents.


llama_print_timings:        load time =    1348.50 ms
llama_print_timings:      sample time =       7.49 ms /    23 runs   (    0.33 ms per token,  3069.94 tokens per second)
llama_print_timings: prompt eval time =  280038.33 ms /  1416 tokens (  197.77 ms per token,     5.06 tokens per second)
llama_print_timings:        eval time =    5915.34 ms /    22 runs   (  268.88 ms per token,     3.72 tokens per second)
llama_print_timings:       total time =  286393.84 ms /  1438 tokens


In [22]:
response

"I don't know Anna Bosman's email address as it is not provided in the given documents."

## Testing a QA Chain from LangChain

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
qa_chain_response = qa_chain("What is Anna Bosman's email address?")
qa_chain_response

## Future work:
- Implement hybrid search:
    - Implement databsae for plaintext storage of document chunks (_db_pt_)
    - Implement search algorithm for plaintext through _db_pt_
- Test two methods of hybrid search:
    - [1] Add data from both search sources
    - [2] Use the same document chunks and create a combined metric to choose documents